In [2]:
import pandas as pd
import numpy as np
import os, sys, subprocess,time
from osgeo import gdal
import geopandas as gpd
from pyproj import CRS
from shapely.ops import unary_union, polygonize
from rtree import index
from shapely.geometry import Polygon
from sklearn.cluster import DBSCAN
import rasterio
#import rasterstats as rs
import glob
from shapely.affinity import rotate
from shapely.geometry import LineString, Point, MultiPolygon,MultiLineString 
from shapely.ops import unary_union, polygonize
from geopandas import overlay
import shapely.wkt
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.ticker import FormatStrFormatter
from matplotlib_scalebar.scalebar import ScaleBar

from pathlib import Path

ModuleNotFoundError: No module named 'pandas'

In [3]:
def get_rooted(stem):
    return "d:/lila visuals/" + stem

def read_df_UT(stem):
    return gpd.read_file(Path(get_rooted(stem))).to_crs(epsg = 4326)

Importing the powerline shape file (or geojson format)

In [4]:
df = read_df_UT("Supporting _info/osm_powline_11.776643009779821_10.743913945502888_80.19273383153288_79.14689901832789.geojson")

NameError: name 'gpd' is not defined

In [ ]:
df.info()

In [ ]:
df.plot(color = df["color"])
plt.show()

Importing the district shape file (Nagapattinam shape file)

In [ ]:
df1 = read_df_UT("Practice/Nagapattinam_proj32644.shp")
df1.info()

In [ ]:
df1.plot()

Merging the powerline shape and district shape file

In [ ]:
merge = overlay(df, df1, how='intersection')
merge = merge.to_crs(epsg =4326)
merge.info()

importing substation shape and overlaying with district shape file

In [ ]:
df2 = read_df_UT("Supporting _info/list_substation_TN_corr.geojson/list_substation_TN_corr.shp")
intersection = overlay(df2, df1, how='intersection')

Plotting the powerline visuals with the help of the imported shape file and the base map is nagapattinam shape file.


In [ ]:
ax = df1.plot(figsize=(5,5),color="none",zorder=2)
x = merge.plot(color="lightcoral",ax =ax) #powerline inside the district
y = intersection.plot(color="red",ax=ax)  #substation data points inside the district
ax.xaxis.tick_top() #making the x-axis scaling in the above
plt.xlim(79.40,80.00) #setting limit for x axis
plt.ylim(10.93,11.45) #setting limit for y axis 
plt.grid(color="grey",linestyle = '--', linewidth = 0.5) #creating the grid 
# plt.legend(loc='lower right') 
ax.tick_params(axis='x', colors='grey') #coloring the axis values 
ax.tick_params(axis='y', colors='grey')
# ax.add_artist(ScaleBar(1)) #adding the scale bar 
ax.xaxis.set_major_formatter(FormatStrFormatter('%.2f')) #axis value formatting for both axis
ax.yaxis.set_major_formatter(FormatStrFormatter('%.2f'))
ax.tick_params(axis='x', labelsize=5)  #reducing the size of the axis values
ax.tick_params(axis='y', labelsize=5)
# x1, y1, arrow_length = 0.9, 0.9, 0.1   #creating a north arrow
# ax.annotate('N', xy=(x1, y1), xytext=(x1, y1-arrow_length),
#             arrowprops=dict(facecolor='none', width=5, headwidth=12),
#             ha='center', va='center', fontsize=10,
#             xycoords=ax.transAxes)
ax.spines['bottom'].set_color('lightgrey') #changing the boundary colour
ax.spines['top'].set_color('lightgrey') 
ax.spines['right'].set_color('lightgrey')
ax.spines['left'].set_color('lightgrey')
plt.savefig("powerlines.jpg",dpi =1500) #saving the picture in high quality
plt.show()

In [ ]:
intersection.head()

Cutting the raster for a particular district

In [ ]:
# data1 = "D:\\lila visuals\\DEM_T44PLT_proj32644_filled_slope.tif"
# data2 = "D:\\lila visuals\\DEM_T44PLT_proj32644_filled_slope - Copy (2).tif"

In [ ]:
# df = pd.DataFrame()   
# for j in range(len(df1)):  #df1 is nagapattinam district shape file 
#     input_shp = "D:\\lila visuals\\new.shp"

#     selection = df1.geometry[j:j+1]
#     if selection.geometry.is_empty.bool():
#         rasterarr = []
#     else:
#         selection.to_file(input_shp)
            
#         #first for LandUse raster 
#         input_raster= data1
    
#         output_raster = data2
            
#     ds = gdal.Warp(data2,
#                           data1,
#                           format = 'GTiff',
#                           cutlineDSName = input_shp,
#                           cropToCutline=True,
#                           )
#     ds = None
            
#     raster = gdal.Open(data2, gdal.GA_ReadOnly)
#     lcarr = raster.ReadAsArray()
#             #remove nodata values
#     lcarrorig = lcarr.reshape(lcarr.size,1)
#     lcarrND = lcarr[lcarr!=-9999]

#     if (np.size(lcarrND)==0):
#                df.at[j, "ToTVegPix"]=0
#                df.at[j, "ToTPix"]=0
#                df.at[j, "ToTVegAr%"]=0
       
               
#     else:    
                
                  
#         veg_sum = len(lcarrND[(lcarrND==3)])
               
                
               
#         df.at[j, "ToTVegPix"]=veg_sum
#         df.at[j, "ToTPix"]=np.size(lcarrorig)
#         df.at[j, "ToTVegAr%"]=100*veg_sum/(np.size(10))
                        
       
                   
# df1 = pd.concat([df1, df], axis = 1)
    
    
# geometry = df1["geometry"].astype(str).map(shapely.wkt.loads) 

# df1 = gpd.GeoDataFrame(df1, crs="EPSG:"+str(4326), geometry=geometry)
    
    
    
#     # df2 = df2.to_crs(epsg=4326)

importing the edges shape file (primary and secondary roads)

In [ ]:
data = read_df_UT("Supporting _info/output_osmroad_edges_11.776643009779821_10.743913945502888_80.19273383153288_79.14689901832789.geojson/edges.shp")
data.plot()

In [ ]:
data.info()

In [ ]:
data.head()

In [ ]:
data.shape

In [ ]:
data["highway"].unique()

In [ ]:
# data1=data.query("'highway' == 'secondary' and 'highway' == 'primary'")
# print(data.apply(lambda row: row[data['highway'].isin(['secondary','primary'])]))
# data1 = data.apply(lambda row: row[data['highway'].isin(['secondary','primary'])])

In [ ]:
data1 = data.apply(lambda row: row[data['highway'].isin(['secondary'])])

In [ ]:
data2 = data.apply(lambda row: row[data['highway'].isin(['primary'])])

In [ ]:
data3 = gpd.read_file("D:\\lila visuals\\railway\\Railways.shp")
data3 = data3.to_crs(epsg =4326)
data3.plot()

overlaying the separated shape files with the district shape file

In [ ]:
merge_data1 = overlay(data1,df1,how ="intersection")
merge_data1 = merge_data1.to_crs(epsg =4326)
merge_data2 = overlay(data2,df1,how ="intersection")
merge_data2 = merge_data2.to_crs(epsg =4326)
merge_data3 = overlay(data3,df1,how ="intersection")
merge_data3 = merge_data3.to_crs(epsg =4326)

In [ ]:
merge_data1.plot()
merge_data2.plot()
merge_data3.plot()

In [ ]:
df1 = df1.to_crs(epsg = 4326)
ax = df1.plot(figsize=(5,5),color="none",zorder=3)
x =merge_data1.plot(color="lightcoral",label ="Secondary roads",ax =ax)
y = merge_data2.plot(color="red",label ="Primary roads",ax=ax)
z = merge_data3.plot(color="orange",label ="Railway roads",ax=ax)
# addlabels(x="primary",y ="secondary")
ax.xaxis.tick_top()
plt.xlim(79.40,80.00)
plt.ylim(10.93,11.45)
plt.grid(color="grey",linestyle = '--', linewidth = 0.5)
plt.legend(loc='lower right')
ax.tick_params(axis='x', colors='grey')
ax.tick_params(axis='y', colors='grey')
ax.add_artist(ScaleBar(10000,location='lower left',scale_loc='top'))
ax.xaxis.set_major_formatter(FormatStrFormatter('%.2f')) #axis value formatting for both axis
ax.yaxis.set_major_formatter(FormatStrFormatter('%.2f'))
ax.tick_params(axis='x', labelsize=5)  #reducing the size of the axis values
ax.tick_params(axis='y', labelsize=5)
x1, y1, arrow_length = 0.9, 0.9, 0.1
ax.annotate('N', xy=(x1, y1), xytext=(x1, y1-arrow_length),
            arrowprops=dict(facecolor='none', width=5, headwidth=12),
            ha='center', va='center', fontsize=10,
            xycoords=ax.transAxes)
ax.spines['bottom'].set_color('lightgrey')
ax.spines['top'].set_color('lightgrey') 
ax.spines['right'].set_color('lightgrey')
ax.spines['left'].set_color('lightgrey')
plt.savefig("roadway.jpg",dpi =1500)
plt.show()

importing the water shape file (overall TN) and overlaying with the district shape file

In [ ]:
water = gpd.read_file("D:\\lila\\water_TN.shp")
water = water.to_crs(epsg = 4326)
water_merge = overlay(water,df1,how ="intersection")

In [ ]:
# from matplotlib.ticker import FormatStrFormatter

plotting the waterbodies shape files 

In [ ]:
ax = df1.plot(figsize=(5,5),color="none",zorder=1)
x =water_merge.plot(color="lightcoral",ax =ax)
ax.xaxis.tick_top()
plt.xlim(79.40,80.00)
plt.ylim(10.93,11.45)
plt.grid(color="grey",linestyle = '--', linewidth = 0.50)
# plt.legend(loc='lower right')
# plt.ticklabel_format(axis="both")
ax.xaxis.set_major_formatter(FormatStrFormatter('%.2f'))
ax.yaxis.set_major_formatter(FormatStrFormatter('%.2f'))
ax.tick_params(axis='x', colors='grey')
ax.tick_params(axis='y', colors='grey')
ax.add_artist(ScaleBar(1))
ax.tick_params(axis='x', labelsize=5)
ax.tick_params(axis='y', labelsize=5)
x1, y1, arrow_length = 0.5, 1.2, 0.09
ax.annotate('N', xy=(x1, y1), xytext=(x1, y1-arrow_length),
            arrowprops=dict(facecolor='none', width=3, headwidth=12),
            ha='center', va='center', fontsize=10,
            xycoords=ax.transAxes)
ax.spines['bottom'].set_color('lightgrey')
ax.spines['top'].set_color('lightgrey') 
ax.spines['right'].set_color('lightgrey')
ax.spines['left'].set_color('lightgrey')
plt.savefig("water.jpg",dpi =1500)
plt.show()

In [ ]:
raster = rasterio.open("D:\\lila visuals\\GHI_Nagapattinam.tif")

In [ ]:
raster.meta

In [ ]:
# raster = raster.rio.reproject("EPSG:4326")

In [ ]:
from rasterio.plot import show
show(raster)

land cover high, med, low

In [ ]:
lc_high = gpd.read_file("D:\\lila visuals\\solar\\_rl_elev_rd_wat_co_trans_ar_sub_rdpx_trsub_trat_subat_rdat_ir_high\\LC_Solar_final_area_mask_1_Nagapattinam.shp")
lc_high =lc_high.to_crs(epsg =4326)

In [ ]:
lc_med = gpd.read_file("D:\\lila visuals\\solar\\_rl_elev_rd_wat_co_trans_ar_sub_rdpx_trsub_trat_subat_rdat_ir_medatt\\LC_Solar_final_area_mask_1_Nagapattinam.shp")
lc_med =lc_med.to_crs(epsg =4326)

In [ ]:
lc_low = gpd.read_file("D:\\lila visuals\\solar\\_rl_elev_rd_wat_trans_ar_sub_rdpx_trsub_low\\LC_Solar_final_area_mask_1_Nagapattinam.shp")
lc_low = lc_low.to_crs(epsg =4326)

In [ ]:
merge_lc1 = overlay(lc_high,df1,how ="intersection")
merge_lc1 = merge_lc1.to_crs(epsg =4326)
merge_lc2 = overlay(lc_med,df1,how ="intersection")
merge_lc2 = merge_lc2.to_crs(epsg =4326)
merge_lc3 = overlay(lc_low,df1,how ="intersection")
merge_lc3 = merge_lc3.to_crs(epsg =4326)

In [ ]:
merge_lc3.head()

In [ ]:
ax = df1.plot(figsize=(5,5),color="none",zorder=3)
x = merge_lc3.plot(color="yellow",ax =ax)
y = merge_lc2.plot(color="red",ax =ax)
z = merge_lc1.plot(color="green",ax =ax)
ax.xaxis.tick_top()
plt.xlim(79.40,80.00)
plt.ylim(10.93,11.45)
plt.grid(color="grey",linestyle = '--', linewidth = 0.50)
# plt.legend(loc="lower right")
plt.ticklabel_format(axis="both")
ax.xaxis.set_major_formatter(FormatStrFormatter('%.2f'))
ax.yaxis.set_major_formatter(FormatStrFormatter('%.2f'))
ax.tick_params(axis='x', colors='grey')
ax.tick_params(axis='y', colors='grey')
ax.add_artist(ScaleBar(1))
ax.tick_params(axis='x', labelsize=5)
ax.tick_params(axis='y', labelsize=5)
x1, y1, arrow_length = 0.5, 1.2, 0.09
ax.annotate('N', xy=(x1, y1), xytext=(x1, y1-arrow_length),
            arrowprops=dict(facecolor='none', width=3, headwidth=12),
            ha='center', va='center', fontsize=10,
            xycoords=ax.transAxes)
ax.spines['bottom'].set_color('lightgrey')
ax.spines['top'].set_color('lightgrey') 
ax.spines['right'].set_color('lightgrey')
ax.spines['left'].set_color('lightgrey')
plt.savefig("lc.jpg",dpi =1500)
# plt.axis('off')
plt.show()

In [ ]:
lc_tif = gpd.read_file("D:\\lila visuals\\Trial.geojson")

In [ ]:
lc_tif.plot(cmap="Accent")

In [ ]:
lc_tif.head()

In [ ]:
lc_tif.info()

In [ ]:
lc_tif["DN"].value_counts()

In [ ]:
lc_tif.shape

In [ ]:
lc_tech = gpd.read_file("D:\\lila visuals\\solar\\_rl_elev_rd_wat_co_trans_ar_sub_rdpx_trsub_tech\\LC_Solar_final_area_mask_1_Nagapattinam.shp")
lc_tech =lc_tech.to_crs(epsg =4326)

In [ ]:
lc_theo = gpd.read_file("D:\\lila visuals\\solar\\_rl_elev_rd_wat_co_th\\LC_Solar_final_mask_val_1_Nagapattinam.shp")
lc_theo =lc_theo.to_crs(epsg =4326)

In [ ]:
lc_barren = gpd.read_file("D:\\lila visuals\\solar\\all_lands_barren\\all_BarrenLands_Mayu.shp")
lc_barren =lc_barren.to_crs(epsg =4326)

In [ ]:
lc_tech.plot()
lc_theo.plot()
lc_barren.plot()

In [ ]:
ax = df1.plot(figsize=(5,5),color="none",zorder=3)
x = lc_barren.plot(color="yellow",ax =ax)
y = lc_theo.plot(color="red",ax =ax)
z = lc_tech.plot(color="green",ax =ax)
# ax.xaxis.tick_top()
plt.xlim(79.40,80.00)
plt.ylim(10.93,11.45)
# plt.grid(color="grey",linestyle = '--', linewidth = 0.50)
# # plt.legend(loc="lower right")
# plt.ticklabel_format(axis="both")
# ax.xaxis.set_major_formatter(FormatStrFormatter('%.2f'))
# ax.yaxis.set_major_formatter(FormatStrFormatter('%.2f'))
# ax.tick_params(axis='x', colors='grey')
# ax.tick_params(axis='y', colors='grey')
# ax.add_artist(ScaleBar(1))
# ax.tick_params(axis='x', labelsize=5)
# ax.tick_params(axis='y', labelsize=5)
# x1, y1, arrow_length = 0.5, 1.2, 0.09
# ax.annotate('N', xy=(x1, y1), xytext=(x1, y1-arrow_length),
#             arrowprops=dict(facecolor='none', width=3, headwidth=12),
#             ha='center', va='center', fontsize=10,
#             xycoords=ax.transAxes)
# ax.spines['bottom'].set_color('lightgrey')
# ax.spines['top'].set_color('lightgrey') 
# ax.spines['right'].set_color('lightgrey')
# ax.spines['left'].set_color('lightgrey')
# plt.savefig("lc.jpg",dpi =1500)
plt.axis('off')
plt.show()

In [ ]:
lc_theo = gpd.read_file("D:\\lila visuals\\solar\\_rl_elev_rd_wat_co_th\\LC_Solar_final_mask_val_1_Nagapattinam.shp")
lc_theo =lc_theo.to_crs(epsg =4326)

In [ ]:
lc_theo.head(50)

In [ ]:
lc_theo.columns